## Set Strava API user token
Import the Client from the [stravalib library](https://github.com/hozn/stravalib/) and load your Access Token.

You can find your personal token at https://www.strava.com/settings/api. Copy your token to a file named `STRAVA_TOKEN` in the home directory.

In [ ]:
from stravalib import unithelper
from stravalib.client import Client as StravaClient

strava_token = !cat STRAVA_TOKEN
strava_client = StravaClient(access_token=strava_token)

## Import Activity from Strava

In [ ]:
import sys
import pandas as pd
from termcolor import colored

# Zermatt Westside https://homebytwo.ch/routes/81/
route = pd.read_hdf('zermatt_westside.h5')
activity = strava_client.get_activity(1751305566) # replace with your data

# get activity and streams
stream_types = ['latlng', 'time', 'altitude', 'distance']
raw_streams = strava_client.get_activity_streams(activity.id, types=stream_types, resolution='high')

# Display results
date = activity.start_date.strftime('%d.%m.%y')    
message_title = "Successfuly importered {}\n".format(activity.name,)
message_body = "{} on {}: {}, +{}".format(
    activity.type,
    date,
    unithelper.kilometers(activity.distance),
    unithelper.meters(activity.total_elevation_gain),
)

sys.stdout.write(colored(message_title, attrs=['bold']))
sys.stdout.write(message_body)

## Put activity data in Pandas DataFrame

In [ ]:
data = pd.DataFrame()

for key, stream in raw_streams.items():
    data[key] = stream.data
data.head()


## Import Route with Schedule from Homebytwo and calculate schedule

In [ ]:
# Set parameters
slope_squared_param = 4.03368561
slope_param = 1.22328986
flat_param = 0.3210731982071581
total_elevation_gain_param = 0.000133084109

# Calculate elevation gain and distance
route['distance'] = route['length'].diff().fillna(value=0)
route['gain'] = route['altitude'].diff().fillna(value=0)

# Calculate schedule
route['schedule'] = (
    (slope_squared_param * route['gain']**2) / route['distance']
    + slope_param * route['gain']
    + flat_param * route['distance']
    + total_elevation_gain_param * route['totalup'] * route['distance']
).where(route['distance'] > 0.1, 0).cumsum().fillna(value=0)

route.tail()

## Calculate expected schedule from GPS position

In [ ]:
from datetime import timedelta

from pyproj import Proj, transform, Geod
from shapely.geometry import LineString, Point
from numpy import interp
import matplotlib.pyplot as plt
%matplotlib inline

# define projections to transform coordinates and calculate distance
swiss_projection = Proj(init='epsg:21781')
google_projection = Proj(init='epsg:4326')
geod = Geod(ellps='WGS84')

# create route LineString
coords = route[['lat', 'lng']]
swiss_route_coords = [tuple(x) for x in coords.values]
route_coords = [transform(swiss_projection, google_projection, coords[0], coords[1]) for coords in swiss_route_coords]
route_coords = [(coords[1], coords[0]) for coords in route_coords]
route_linestring = LineString(route_coords)
route_length = route['length'].max()

def get_expected_schedule(latlng):
    activity_point = Point(latlng)
    line_location = route_linestring.project(activity_point, normalized=True)
    interp_x = line_location * route_length
    return interp(interp_x, route['length'], route['schedule'])

def get_distance_from_line(latlng):
    activity_point = Point(latlng)
    line_location = route_linestring.project(activity_point, normalized=True)
    route_point = route_linestring.interpolate(line_location, normalized=True)
    az12, az21, distance = geod.inv(activity_point.x, activity_point.y, route_point.x, route_point.y)
    return distance

data['expected_schedule'] = data['latlng'].apply(get_expected_schedule)
data['distance_from_route'] = data['latlng'].apply(get_distance_from_line)
data['time_diff'] = data['time'] - data['expected_schedule']

fig = plt.figure(figsize=(40, 10))
ax = fig.add_subplot(122)
ax.plot(data['distance'], data['time_diff'], 'b-')
ax.plot(data['distance'], data['distance_from_route'], 'r-')
ax.plot(data['distance'], data['altitude'], 'g')

data.tail()
